In [ ]:
# 라이브러리 설치 
# !pip install gensim

In [2]:
from gensim.models import Word2Vec

In [3]:
# 토큰화가 된 데이터 셋을 임의로 생성 
sentences = [
    ['오늘', '날씨', '좋다'], 
    ['강아지', '산책', '기분', '좋다'], 
    ['비', '오다', '산책', '못하다']
]

In [4]:
# Word2Vec 객체를 생성하면서 데이터를 입력 
model = Word2Vec(
    sentences= sentences, 
    vector_size= 100, 
    window=5, 
    min_count=1, 
    sg= 1, 
    epochs= 100, 
    seed = 42 
)

In [ ]:
# 단위 벡터 출력 
print(model.wv['산책'])

KeyedVectors<vector_size=100, 9 keys>


In [7]:
# 코사인 유사도 
print(model.wv.most_similar('산책'))

[('오늘', 0.1943289339542389), ('좋다', 0.09307777136564255), ('기분', 0.03429308161139488), ('못하다', 0.029127657413482666), ('날씨', 0.00435193907469511), ('오다', -0.007374857552349567), ('강아지', -0.009019400924444199), ('비', -0.02072860859334469)]


### Word2Vec 
- 문자를 수치형으로 변환 시켜주는 딥러닝 기반의 임베딩 기술

- 주요 매개변수 
    - sentences
        - 기본값 : None
        - 토큰화가 된 문장 데이터 ( 2차원 리스트의 형태 )
        - 문장의 데이터가 None 기본값이라면 -> 학습을 하는 메서드가 존재할것이다. 
    - vector_size
        - 기본값 : 100
        - 임베딩 백터 차원의 개수 (feature 수)
    - window
        - 기본값 : 5
        - 예측 시 고려할 주변 단어와의 거리 (문맥의 크기)
    - sg
        - 기본값 : 0
        - 0 인 경우 CBOW 방식(주변 단어를 이용하여 중심 단어를 예측)
        - 1 인 경우 Skip-Gram 방식(중심 단어를 이용하여 주변 단어를 예측)
        - 빠른 계산이 필요한 경우라면 0
        - 일반적인 경우에는 1은 선택
    - min_count
        - 기본값 : 5
        - 최소 등장 빈도 수 
        - 적게 등장하는 단어들을 제외
    - hs 
        - 기본값 : 0
        - 계산 횟수 지정 
        - 0 : Negative Sampling (계산량 적음)
        - 1 : Hierarchical Softmax ( 계산량 많음 )
    - epochs
        - 기본값 : 100
        - 반복 학습하는 횟수 지정 
    - max_vocab_size
        - 기본값 : None
        - 메모리 제한 시 사용할 최대 단어의 개수 
- 주요 속성 
    - wv
        - 학습된 단어 백터( 객체 형으로 출력 )
        - 예시 :  model.wv['단어']
    - wv.index_to_key
        - 단어의 리스트(학습이 된 단어의 개수 (최소 등장 횟수에 영향))
        - 등장 빈도수에 따라 자동 정렬 
    - wv.key_to_index
        - 단어 -> 인덱스로 매칭 
        - 특정 단어가 인덱스 몇에 위치하는가
    - copus_total_word
        - 전체 학습이 된 단어의 개수 
    - epochs
        - 학습 epoch의 수 
    - vector_size
        - 백터 차원의 수 
- 메서드 
    - wv.most_similar( {word}, topn = 10 )
        - 특정 단어와 유사한 단어를 출력 
        - topn은 유사한 단어의 개수 
    - wv.similarity({word1}, {word2})
        - 두 단어 간의 코사인 유사도 
    - wv.get_vector( {word} )
        - 특정 단어의 벡터를 반환
    - train( ... )
        - 추가 데이터로 학습 
    - save( {path} )
        - 모델 자체를 파일로 저장 
    - Word2Vec.load()
        - 모델을 불러오기 

In [1]:
# 토큰화 (konlpy)
# 백터화 (Word2Vec)
# 입력데이터가 생성이 되었으니 머신러닝 모델에 학습 

In [4]:
import numpy as np 
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Komoran

In [5]:
X_raw = [
    '오늘 날씨가 좋다 여행 가고 싶다',
    '기온이 너무 떨어져서 아무것도 하기 싫다', 
    '음식이 정말 맛있었고 서비스도 좋았다', 
    '완전 별로고 돈이 아깝다 다시는 안 온다', 
    '수업이 너무 지루하고 졸리다', 
    '영화가 너무 재미있어서 시간이 가는 줄 몰랐다'
]
target = [1, 0, 1, 0, 0, 1]

In [7]:
# 토큰화 작업 
komoran = Komoran()
# 사용할 형태들을 지정 
allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'SL', 'MAG']

tokens = []
for raw in X_raw:
    words = []
    for word, pos in komoran.pos(raw):
        if pos in allow_pos:
            words.append(word)
    tokens.append(words)

tokens

[['오늘', '날씨', '좋', '여행', '가'],
 ['기온', '너무', '떨어지', '아무것', '하', '싫'],
 ['음식', '정말', '맛있', '서비스', '좋'],
 ['완전', '별로', '돈', '아깝', '다시', '안', '오'],
 ['수업', '너무', '졸리'],
 ['영화', '너무', '재미있', '시간', '가', '모르']]

In [8]:
# Word2Vec을 이용하여 학습(Skip-gram 방식)
w2v = Word2Vec(
    sentences= tokens, 
    vector_size= 100, 
    window = 5, 
    min_count= 1, 
    sg = 1, 
    epochs= 100, 
    seed = 42, 
    workers=2
)

In [ ]:
# w2v 객체에서 wv 속성을 자주 사용하기에 변수에 따로 저장 
wv = w2v.wv

In [ ]:
# tokens의 단어들 중  w2v의 index_to_key에 존재하는 데이터의 단위 백터를 확인 
vectors = []
for token in tokens:
    for word in token:
        # tokens의 단어들 중 Word2vec에서 학습 단어들에 포함되어있다면 
        vec = []
        if word in wv.index_to_key:
            # 해당 단어의 단어 백터를 출력하여 vertors의 대입 
            vec.append(
                wv[word]
            )
    vectors.append(np.mean(vec, axis=0))
vectors

In [35]:
np.array(vectors).shape

(6, 100)

In [36]:
# vectors의 데이터의 의미 -> 독립 변수 
# target 데이터 -> 종속 변수 
svc = SVC(random_state=42)

In [37]:
svc.fit(vectors, target)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [ ]:
# 토큰화 함수 생성 
# komoran 사용 (konlpy 설치가 되어있는 경우)
# 설치가 되어있지 않은 경우에는 split()을 이용하여 토큰화 
def build_tokenize():
    try:
        from konlpy.tag import Komoran
        komoran = Komoran()
        allow_pos = ['NNP', 'NNG', 'VV', 'VA', 'SL', 'MAG']
        